In [5]:
url

'https://ng.soccerway.com/matches/2023/04/09'

In [16]:
import requests
import bs4 as bs
import pandas as pd
from datetime import date
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
import csv


#input date to get fixtures from
matchDay = input('Input Match Date (YYYY/MM/DD):')

#assign vaiable to url
url = 'https://ng.soccerway.com/matches/'+matchDay
matchD = matchDay.replace('/','-')
#assign 'headers' device and browser illusion
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}


#download html of site
page = requests.get(url, headers=headers)

#create object from downloaded html
soup = bs.BeautifulSoup(page.text, 'html.parser')

#find the 'a' tags ins all 'h2' tags
links = [l.find('a') for l in soup.find_all('h2')]

#remove all none items
links = [l for l in links if l != None]
#get all hrefs
links = [l.get('href') for l in links]

#get all items bearing 'national/'
links = [l for l in links if 'national/' in l]

#convert href to links
links = ['https://ng.soccerway.com/'+l for l in links]


#links = list of all competition links in matchDay's page

num0 = int(input('Input 0 to start from beginning : '))
if num0 == 0:
    
    
    #create webdriver object for Edge Browser
    options=Options()
    options.set_preference('permissions.default.image', 2)
    options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    driver = webdriver.Firefox(options=options)
    #maximize window
    driver.maximize_window()
    
    
    num077 = 1
    num07 = num077 - 1
    driver.get(url)
    columns = ['COUNTRY','HOME', 'AWAY','COMP','HOME_RANK', 'AWAY RANK', 'TABLE NO', 'HOME_FORM','AWAY_FORM','HOME_GEN_FORM','H2H_FORM','WIN_PERCENT','TABLE DIFF',' ','HOME_GF','AWAY_GA', 'HOME_GA','AWAY_GF','H2H_GOALS', '1 GOAL_PERCENT', '2HOME_GF','2AWAY_GA', '2HOME_GA','2AWAY_GF','2H2H_GOALS','2 GOAL_PERCENT']
    with open(f'Match Analysis {matchD}.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerow(columns)
    
    
    num00 = 1
    for a in links:
        driver.find_element('xpath', f'//*[@id="livescores"]/div[2]/div[{num00}]/h2/button').click()
        driver.implicitly_wait(2)
        num00+=1
    matches = driver.find_elements(By.TAG_NAME, "a")
    fixturelinks =[]
    for l in matches:
        fixturelinks.append(l.get_attribute('outerHTML'))
        driver.implicitly_wait(3)
    fixturelinks = [l for l in fixturelinks if matchDay in l]
    fixturelinks = [l.split('"')[1] for l in fixturelinks if 'match-preview' in l]
    fixturelinks = ['https://ng.soccerway.com/'+l for l in fixturelinks]
    fixturelinks = [l.replace('#match-preview','') for l in fixturelinks]
    file = open(f'fixtures {matchD}.txt','w')
    for fixt in fixturelinks:
        file.write(fixt+'\n')
    file.close()

    with open(f'fixtures {matchD}.txt','r') as fixtures:
        fixturelinks = fixtures.readlines()
        
          
    fixturelinks = [l.replace('\n','') for l in fixturelinks]
    print('TOTAL MATCHES FOUND = '+str(len(fixturelinks)))

  
    for match3 in fixturelinks:
        #driver.get(match3)

        LgTablePg = requests.get(match3, headers=headers)
        
        country = match3.split('/')[8].upper() #country
        compname = match3.split('/')[9].replace('-',' ').upper() #league
        
        try:
            Lgtable = pd.read_html(LgTablePg.text, match = 'MP')[0]
            #find elements with class name = team-title
            #teams = driver.find_elements(By.CLASS_NAME, "team-title")
            soup = bs.BeautifulSoup(LgTablePg.content, 'html.parser')
            teAms = soup.find_all("a", {"class":"team-title"})
            
            #get html attribute of found element for home team
            #homeLink = teams[0].get_attribute('outerHTML').split('"')[3]
            homeLink = teAms[0].get('href')
            
            #get html attribute of found element for away team
            #awayLink = teams[1].get_attribute('outerHTML').split('"')[3]
            awayLink = teAms[1].get('href')
            
            #convert to matches pagelink of home and away teams
            homeLink = 'https://ng.soccerway.com'+homeLink+'/matches/'
            awayLink = 'https://ng.soccerway.com'+awayLink+'/matches/'

            #get html of home matches page
            homeLinkH = requests.get(homeLink, headers=headers)
            time.sleep(1)
            #read the first table of the html
            homeDF = pd.read_html(homeLinkH.text)[0]

            #rename columns
            homeDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #rewrite matchDay from yyyy/mm/dd to dd/mm/yy
            dayMatch = matchDay[8:]+matchDay[4:8]+matchDay[2:4]

            #create homeDF1 which is all rows bearing dayMatch as its Date data
            homeDF1 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH
            date_indexH = int(list(homeDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName = homeDF1.loc[date_indexH,'Competition']

            #home & away team names
            homeName = homeDF1.loc[date_indexH,'Home']
            awayName = homeDF1.loc[date_indexH,'Away']

            #cleaning homeDF

            #drop all rows except one with Competition column data = compName 
            homeDF = homeDF[homeDF['Competition'] == compName]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #create homeDF2 which is all rows bearing dayMatch as its Date data
            homeDF2 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH2
            date_indexH2 = int(list(homeDF2['Date'].index)[0])

            #drop all rows including and after matchDay
            homeDF = homeDF.iloc[:date_indexH2]

            #drop all columns except specificied
            homeDF = homeDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_list = homeDF['Score'].values.tolist()
            score_list1 = []
            for a in score_list:
                try:
                    eval(a)
                except:
                    score_list1.append(None)
                else:
                    score_list1.append(eval(a))
            homeDF['Points'] = score_list1

            #reverse rows in homeDF
            homeDF = homeDF.iloc[::-1]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #DF for last 5 matches where home team played at home
            homeDF_home = homeDF[homeDF['Home']==homeName].head(5)

            #DF for last 5 macthes for home
            homeDF_Gen = homeDF.head(5)

            #DF for last 5 matches where home team played at away
            homeDF_away = homeDF[homeDF['Away']==homeName].head(5)


            #homeName = name of home team
            #awayName = name of away team
            #homeDF = DF for all matches for home team
            #compName = Competiton name home POV
            #homeDF_home = home's home analysis
            #homeDF_away = home's away analysis

            #get html of away matches page
            awayLinkH = requests.get(awayLink, headers=headers)

            #read the first table of the html
            awayDF = pd.read_html(awayLinkH.text)[0]

            #rename columns
            awayDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #create awayDF1 which is all rows bearing dayMatch as its Date data
            awayDF1 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA = int(list(awayDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName2 = awayDF1.loc[date_indexA,'Competition']

            #drop all rows except one with Competition column data = compName 
            awayDF = awayDF[awayDF['Competition'] == compName2]

            #reset index
            awayDF = awayDF.reset_index(drop=True)

            #create awayDF2 which is all rows bearing dayMatch as its Date data
            awayDF2 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA2 = int(list(awayDF2['Date'].index)[0])

            #drop all rows including and after macthDay
            awayDF = awayDF.iloc[:date_indexA2]  

            #drop all columns except specificied
            awayDF = awayDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_listA = awayDF['Score'].values.tolist()
            score_listA1 = []
            for a in score_listA:
                try:
                    eval(a)
                except:
                    score_listA1.append(None)
                else:
                    score_listA1.append(eval(a))
            awayDF['Points'] = score_listA1

            #reverse rows in awayDF
            awayDF = awayDF.iloc[::-1]

            #DF for last 5 matches where away team played at home
            awayDF_home = awayDF[awayDF['Home']==awayName].head(5)

            #DF for last 5 matches where away team played at away
            awayDF_away = awayDF[awayDF['Away']==awayName].head(5)


            #awayName = name of away team
            #compName2 = competition name away POV
            #awayDF = DF for matches played by away team
            #awayDF_home = DF for away's home analysis
            #awayDF_away = DF for away's away analysis



            #evaluate home team's home form 

            #convert 'Points' homeDF_home columns to a list
            HPointsList = homeDF_home['Points'].values.tolist()

            #counter
            num001 = 0
            #iterate across list, and change point data into correct point value
            for point in HPointsList:
                if point > 0:
                    HPointsList[num001] = 1
                elif point < 0:
                    HPointsList[num001] = 0
                elif point == 0:
                    HPointsList[num001] = 0.5
                else:
                    HPointsList[num001] = 0
                num001+=1
            #calculate homeform value
            home_form_value = sum(HPointsList)

            #evaluate away team's away form

            #convert 'Points' awayDF_away columns to a list
            APointsList = awayDF_away['Points'].values.tolist()

            #counter
            num003 = 0
            #iterate across list, and change point data into correct point value
            for point in APointsList:
                if point > 0:
                    APointsList[num003] = 1
                elif point < 0:
                    APointsList[num003] = 0
                elif point == 0:
                    APointsList[num003] = 0.5
                else:
                    APointsList[num003] = 0
                num003+=1
            #calculate awayform value
            away_form_value = sum(APointsList)

            #evaluate home team's general form

            #convert 'Points' homeDF_Gen columns to a list
            HGPointsList = homeDF_Gen['Points'].values.tolist()

            #convert 'Home' homeDF_Gen columns to a list
            homeTeamList = homeDF_Gen['Home'].values.tolist()

            #counter
            num002 = 0
            #iterate across list, and change point data into correct point value
            for point in HGPointsList:
                if homeTeamList[num002] == homeName:
                    if point > 0:
                        HGPointsList[num002] = 1
                    elif point < 0:
                        HGPointsList[num002] = 0
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                else:
                    if point > 0:
                        HGPointsList[num002] = 0
                    elif point < 0:
                        HGPointsList[num002] = 1
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                num002+=1
            #calculate homeGen value
            home_Gen_value = sum(HGPointsList)


            #HOME TEAM GOALS

            #convert homeDF_home 'Score' column into list
            homeGoalList = homeDF_home['Score'].values.tolist()

            #check score lines in above homeGoallist and append 1 point (homeGoalF) to scorelines where home scored
            homeGoalF = []
            homeGoalF5 = []
            for goal in homeGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        homeGoalF.append(1)
                    else:
                        homeGoalF.append(0)
                    if int(goal[0]) > 1:
                        homeGoalF5.append(1)
                    else:
                        homeGoalF5.append(0)
            #sum the appended points to get home_GF
            home_GF = sum(homeGoalF)
            home_GF5 = sum(homeGoalF5)
            #check score lines in above homeGoalList and 
            #append 1 point (homeGoalA) to scorelines where home conceded
            homeGoalA = []
            homeGoalA5 = []
            for goal in homeGoalList[:2]:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        homeGoalA.append(1)
                    else:
                        homeGoalA.append(0)
                    if int(goal[4]) > 1:
                        homeGoalA5.append(1)
                    else:
                        homeGoalA5.append(0)
            #sum the appended points to get home_GA
            home_GA = sum(homeGoalA)
            home_GA5 = sum(homeGoalA5)

            #AWAY TEAM GOALS

            #convert awayDF_away 'Score' column into list
            awayGoalList = awayDF_away['Score'].values.tolist()

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalA) to scorelines where away conceded
            awayGoalA = []
            awayGoalA5 = []
            for goal in awayGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        awayGoalA.append(1)
                    else:
                        awayGoalA.append(0)
                    if int(goal[0]) > 1:
                        awayGoalA5.append(1)
                    else:
                        awayGoalA5.append(0)

            #sum the appended points to get away_GA
            away_GA = sum(awayGoalA)
            away_GA5 = sum(awayGoalA5)           

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalF) to scorelines where away scored
            awayGoalF = []
            awayGoalF5 = []
            for goal in awayGoalList[:2]:
                try: 
                    int(goal)
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        awayGoalF.append(1)
                    else:
                        awayGoalF.append(0)
                    if int(goal[4]) > 0:
                        awayGoalF5.append(1)
                    else:
                        awayGoalF5.append(0)
            #sum the appended points to get away_GF
            away_GF = sum(awayGoalF)
            away_GF5 = sum(awayGoalF5)

            # h2h

            #link to h2h
            dink = match3+'head2head'
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                       'AppleWebKit/537.36 (KHTML, like Gecko) '\
                       'Chrome/75.0.3770.80 Safari/537.36'}

            #request h2h page
            pgaa = requests.get(dink, headers=headers)



            ##the table is arranged in anti-chronology, the recent data is on top
            ##and the  match to be played is the first row
            ##which want two most recent h2h data


            #read html page using pandas call it jon
            jon = pd.read_html(pgaa.text)[0]
            #jon

            #reverse rows in jon
            jon = jon.iloc[::-1]
            #jon

            #reset index
            jon = jon.reset_index(drop=True)
            #jon

            #create jon1 which is all rows bearing dayMatch as its Date data
            jon1 = jon[jon['Date'] == dayMatch]

            #get row number of matchDay
            date_indexJ = int(list(jon1['Date'].index)[0])

            #drop all rows including and after the specified index row
            jon = jon.iloc[:date_indexJ]

            #drop all columns except specificied
            jon = jon[['Home team', 'Score/Time', 'Away team']]

            #rename columns
            jon.rename(columns={'Home team':'Home', 'Score/Time': 'Score','Away team': 'Away'}, inplace=True)

            #drop all rows except last 2 rows
            #jon = jon.tail(2)

            jon = jon.tail(2)
            jon = jon.reset_index(drop=True)
            #jon

            try:
                jon.loc[0, 'Home']
            except:
                h2hs = ['Home', 0, 0]
            else:            
                if jon.loc[0,'Home'] == jon.loc[jon.shape[0] - 1,'Home']:
                    result=[]
                    num002 = 0
                    for a in jon['Score']:
                        try:
                            eval(a)
                        except:
                            result.append(-1)
                        else:                
                            gd = int(eval(jon.loc[num002,'Score']))
                            result.append(gd)               
                        num002+=1
                    num003 = 0
                    for b in result:
                        if b > 0:
                            result[num003] = 1
                        elif b < 0:
                            result[num003] = 0
                        else:
                            result[num003] = 0.5
                        num003+=1
                    hn = jon.loc[0, 'Home']
                    result = sum(result)

                else:
                    result=[]
                    try:
                        eval(jon.loc[0,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[0, 'Score']))
                        result.append(gd)
                    try:
                        eval(jon.loc[1,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[1, 'Score'][::-1]))
                        result.append(gd)
                    num004 = 0           
                    for c in result:
                        if c > 0:
                            result[num004] = 1
                        elif c < 0:
                            result[num004] = 0
                        else:
                            result[num004] = 0.5
                        num004+=1
                    result = sum(result)
                    hn = jon.loc[0, 'Home']

                if hn in homeName:
                    pass
                else:
                    result = 2 - result

                h2hgoalist = []
                h2hgoalist5 = []
                for a in jon['Score']:
                    x = a.replace('-','+')
                    try:
                        eval(x)
                    except:
                        h2hgoalist.append(0)
                    else:
                        if int(eval(x)) >= 2:
                            h2hgoalist.append(1)
                        else:
                            h2hgoalist.append(0)
                        if int(eval(x)) > 2:
                            h2hgoalist5.append(1)                        
                        else:
                            h2hgoalist5.append(0)
                    
                h2h_goalist = sum(h2hgoalist)  
                h2h_goalist5 = sum(h2hgoalist5)
                h2hs = [hn, result, h2h_goalist, h2h_goalist5]



            LgtableList = Lgtable['Team'].values.tolist()
            rank = 0
            rank2 = 0
            for a in LgtableList:
                if a in homeName:
                    rank = LgtableList.index(a) + 1
                elif a in awayName:
                    rank2 = LgtableList.index(a) + 1
                else:
                    pass
            tableRkList = [rank, rank2, len(LgtableList)]


            #goals
            goal_percent1 = int(((home_GF + away_GA + home_GA + away_GF + h2hs[2])/16)*100)
            goal_percent2 = int(((home_GF5 + away_GA5 + home_GA5 + away_GF5 + h2hs[3])/16)*100)

            #home win
            homeWin_percent = int((((((2 * home_form_value) + (2 * away_form_value) + home_Gen_value)/25) + (0.125 * h2hs[1]))/1.25) * 100)

            #table ranking
            TableDiff = int(((tableRkList[1] - tableRkList[0])/(tableRkList[2] - 1))*100)
            space = ' '
        except:
            print(f'Fixture {num077} skipped')
            num077+=1
            continue
        else:
            Lgtable = pd.read_html(LgTablePg.text, match = 'MP')[0]
            #find elements with class name = team-title
            #teams = driver.find_elements(By.CLASS_NAME, "team-title")
            soup = bs.BeautifulSoup(LgTablePg.content, 'html.parser')
            teAms = soup.find_all("a", {"class":"team-title"})
            
            #get html attribute of found element for home team
            #homeLink = teams[0].get_attribute('outerHTML').split('"')[3]
            homeLink = teAms[0].get('href')
            
            #get html attribute of found element for away team
            #awayLink = teams[1].get_attribute('outerHTML').split('"')[3]
            awayLink = teAms[1].get('href')
            
            #convert to matches pagelink of home and away teams
            homeLink = 'https://ng.soccerway.com'+homeLink+'/matches/'
            awayLink = 'https://ng.soccerway.com'+awayLink+'/matches/'

            #get html of home matches page
            homeLinkH = requests.get(homeLink, headers=headers)
            time.sleep(1)

            #read the first table of the html
            homeDF = pd.read_html(homeLinkH.text)[0]

            #rename columns
            homeDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #rewrite matchDay from yyyy/mm/dd to dd/mm/yy
            dayMatch = matchDay[8:]+matchDay[4:8]+matchDay[2:4]

            #create homeDF1 which is all rows bearing dayMatch as its Date data
            homeDF1 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH
            date_indexH = int(list(homeDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName = homeDF1.loc[date_indexH,'Competition']

            #home & away team names
            homeName = homeDF1.loc[date_indexH,'Home']
            awayName = homeDF1.loc[date_indexH,'Away']

            #cleaning homeDF

            #drop all rows except one with Competition column data = compName 
            homeDF = homeDF[homeDF['Competition'] == compName]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #create homeDF2 which is all rows bearing dayMatch as its Date data
            homeDF2 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH2
            date_indexH2 = int(list(homeDF2['Date'].index)[0])

            #drop all rows including and after matchDay
            homeDF = homeDF.iloc[:date_indexH2]

            #drop all columns except specificied
            homeDF = homeDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_list = homeDF['Score'].values.tolist()
            score_list1 = []
            for a in score_list:
                try:
                    eval(a)
                except:
                    score_list1.append(None)
                else:
                    score_list1.append(eval(a))
            homeDF['Points'] = score_list1

            #reverse rows in homeDF
            homeDF = homeDF.iloc[::-1]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #DF for last 5 matches where home team played at home
            homeDF_home = homeDF[homeDF['Home']==homeName].head(5)

            #DF for last 5 macthes for home
            homeDF_Gen = homeDF.head(5)

            #DF for last 5 matches where home team played at away
            homeDF_away = homeDF[homeDF['Away']==homeName].head(5)


            #homeName = name of home team
            #awayName = name of away team
            #homeDF = DF for all matches for home team
            #compName = Competiton name home POV
            #homeDF_home = home's home analysis
            #homeDF_away = home's away analysis

            #get html of away matches page
            awayLinkH = requests.get(awayLink, headers=headers)

            #read the first table of the html
            awayDF = pd.read_html(awayLinkH.text)[0]

            #rename columns
            awayDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #create awayDF1 which is all rows bearing dayMatch as its Date data
            awayDF1 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA = int(list(awayDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName2 = awayDF1.loc[date_indexA,'Competition']

            #drop all rows except one with Competition column data = compName 
            awayDF = awayDF[awayDF['Competition'] == compName2]

            #reset index
            awayDF = awayDF.reset_index(drop=True)

            #create awayDF2 which is all rows bearing dayMatch as its Date data
            awayDF2 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA2 = int(list(awayDF2['Date'].index)[0])

            #drop all rows including and after macthDay
            awayDF = awayDF.iloc[:date_indexA2]  

            #drop all columns except specificied
            awayDF = awayDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_listA = awayDF['Score'].values.tolist()
            score_listA1 = []
            for a in score_listA:
                try:
                    eval(a)
                except:
                    score_listA1.append(None)
                else:
                    score_listA1.append(eval(a))
            awayDF['Points'] = score_listA1

            #reverse rows in awayDF
            awayDF = awayDF.iloc[::-1]

            #DF for last 5 matches where away team played at home
            awayDF_home = awayDF[awayDF['Home']==awayName].head(5)

            #DF for last 5 matches where away team played at away
            awayDF_away = awayDF[awayDF['Away']==awayName].head(5)


            #awayName = name of away team
            #compName2 = competition name away POV
            #awayDF = DF for matches played by away team
            #awayDF_home = DF for away's home analysis
            #awayDF_away = DF for away's away analysis



            #evaluate home team's home form 

            #convert 'Points' homeDF_home columns to a list
            HPointsList = homeDF_home['Points'].values.tolist()

            #counter
            num001 = 0
            #iterate across list, and change point data into correct point value
            for point in HPointsList:
                if point > 0:
                    HPointsList[num001] = 1
                elif point < 0:
                    HPointsList[num001] = 0
                elif point == 0:
                    HPointsList[num001] = 0.5
                else:
                    HPointsList[num001] = 0
                num001+=1
            #calculate homeform value
            home_form_value = sum(HPointsList)

            #evaluate away team's away form

            #convert 'Points' awayDF_away columns to a list
            APointsList = awayDF_away['Points'].values.tolist()

            #counter
            num003 = 0
            #iterate across list, and change point data into correct point value
            for point in APointsList:
                if point > 0:
                    APointsList[num003] = 1
                elif point < 0:
                    APointsList[num003] = 0
                elif point == 0:
                    APointsList[num003] = 0.5
                else:
                    APointsList[num003] = 0
                num003+=1
            #calculate awayform value
            away_form_value = sum(APointsList)

            #evaluate home team's general form

            #convert 'Points' homeDF_Gen columns to a list
            HGPointsList = homeDF_Gen['Points'].values.tolist()

            #convert 'Home' homeDF_Gen columns to a list
            homeTeamList = homeDF_Gen['Home'].values.tolist()

            #counter
            num002 = 0
            #iterate across list, and change point data into correct point value
            for point in HGPointsList:
                if homeTeamList[num002] == homeName:
                    if point > 0:
                        HGPointsList[num002] = 1
                    elif point < 0:
                        HGPointsList[num002] = 0
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                else:
                    if point > 0:
                        HGPointsList[num002] = 0
                    elif point < 0:
                        HGPointsList[num002] = 1
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                num002+=1
            #calculate homeGen value
            home_Gen_value = sum(HGPointsList)


            #HOME TEAM GOALS

            #convert homeDF_home 'Score' column into list
            homeGoalList = homeDF_home['Score'].values.tolist()

            #check score lines in above homeGoallist and append 1 point (homeGoalF) to scorelines where home scored
            homeGoalF = []
            homeGoalF5 = []
            for goal in homeGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        homeGoalF.append(1)
                    else:
                        homeGoalF.append(0)
                    if int(goal[0]) > 1:
                        homeGoalF5.append(1)
                    else:
                        homeGoalF5.append(0)
            #sum the appended points to get home_GF
            home_GF = sum(homeGoalF)
            home_GF5 = sum(homeGoalF5)
            #check score lines in above homeGoalList and 
            #append 1 point (homeGoalA) to scorelines where home conceded
            homeGoalA = []
            homeGoalA5 = []
            for goal in homeGoalList[:2]:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        homeGoalA.append(1)
                    else:
                        homeGoalA.append(0)
                    if int(goal[4]) > 1:
                        homeGoalA5.append(1)
                    else:
                        homeGoalA5.append(0)
            #sum the appended points to get home_GA
            home_GA = sum(homeGoalA)
            home_GA5 = sum(homeGoalA5)

            #AWAY TEAM GOALS

            #convert awayDF_away 'Score' column into list
            awayGoalList = awayDF_away['Score'].values.tolist()

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalA) to scorelines where away conceded
            awayGoalA = []
            awayGoalA5 = []
            for goal in awayGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        awayGoalA.append(1)
                    else:
                        awayGoalA.append(0)
                    if int(goal[0]) > 1:
                        awayGoalA5.append(1)
                    else:
                        awayGoalA5.append(0)

            #sum the appended points to get away_GA
            away_GA = sum(awayGoalA)
            away_GA5 = sum(awayGoalA5)           

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalF) to scorelines where away scored
            awayGoalF = []
            awayGoalF5 = []
            for goal in awayGoalList[:2]:
                try: 
                    int(goal)
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        awayGoalF.append(1)
                    else:
                        awayGoalF.append(0)
                    if int(goal[4]) > 0:
                        awayGoalF5.append(1)
                    else:
                        awayGoalF5.append(0)
            #sum the appended points to get away_GF
            away_GF = sum(awayGoalF)
            away_GF5 = sum(awayGoalF5)

            # h2h

            #link to h2h
            dink = match3+'head2head'
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                       'AppleWebKit/537.36 (KHTML, like Gecko) '\
                       'Chrome/75.0.3770.80 Safari/537.36'}

            #request h2h page
            pgaa = requests.get(dink, headers=headers)



            ##the table is arranged in anti-chronology, the recent data is on top
            ##and the  match to be played is the first row
            ##which want two most recent h2h data


            #read html page using pandas call it jon
            jon = pd.read_html(pgaa.text)[0]
            #jon

            #reverse rows in jon
            jon = jon.iloc[::-1]
            #jon

            #reset index
            jon = jon.reset_index(drop=True)
            #jon

            #create jon1 which is all rows bearing dayMatch as its Date data
            jon1 = jon[jon['Date'] == dayMatch]

            #get row number of matchDay
            date_indexJ = int(list(jon1['Date'].index)[0])

            #drop all rows including and after the specified index row
            jon = jon.iloc[:date_indexJ]

            #drop all columns except specificied
            jon = jon[['Home team', 'Score/Time', 'Away team']]

            #rename columns
            jon.rename(columns={'Home team':'Home', 'Score/Time': 'Score','Away team': 'Away'}, inplace=True)

            #drop all rows except last 2 rows
            #jon = jon.tail(2)

            jon = jon.tail(2)
            jon = jon.reset_index(drop=True)
            #jon

            try:
                jon.loc[0, 'Home']
            except:
                h2hs = ['Home', 0, 0]
            else:            
                if jon.loc[0,'Home'] == jon.loc[jon.shape[0] - 1,'Home']:
                    result=[]
                    num002 = 0
                    for a in jon['Score']:
                        try:
                            eval(a)
                        except:
                            result.append(-1)
                        else:                
                            gd = int(eval(jon.loc[num002,'Score']))
                            result.append(gd)               
                        num002+=1
                    num003 = 0
                    for b in result:
                        if b > 0:
                            result[num003] = 1
                        elif b < 0:
                            result[num003] = 0
                        else:
                            result[num003] = 0.5
                        num003+=1
                    hn = jon.loc[0, 'Home']
                    result = sum(result)

                else:
                    result=[]
                    try:
                        eval(jon.loc[0,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[0, 'Score']))
                        result.append(gd)
                    try:
                        eval(jon.loc[1,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[1, 'Score'][::-1]))
                        result.append(gd)
                    num004 = 0           
                    for c in result:
                        if c > 0:
                            result[num004] = 1
                        elif c < 0:
                            result[num004] = 0
                        else:
                            result[num004] = 0.5
                        num004+=1
                    result = sum(result)
                    hn = jon.loc[0, 'Home']

                if hn in homeName:
                    pass
                else:
                    result = 2 - result
                    
                h2hgoalist = []
                h2hgoalist5 = []
                for a in jon['Score']:
                    x = a.replace('-','+')
                    try:
                        eval(x)
                    except:
                        h2hgoalist.append(0)
                    else:
                        if int(eval(x)) >= 2:
                            h2hgoalist.append(1)
                        else:
                            h2hgoalist.append(0)
                        if int(eval(x)) > 2:
                            h2hgoalist5.append(1)                        
                        else:
                            h2hgoalist5.append(0)
                    
                h2h_goalist = sum(h2hgoalist)  
                h2h_goalist5 = sum(h2hgoalist5)
                h2hs = [hn, result, h2h_goalist, h2h_goalist5]

            LgtableList = Lgtable['Team'].values.tolist()
            rank = 0
            rank2 = 0
            for a in LgtableList:
                if a in homeName:
                    rank = LgtableList.index(a) + 1
                elif a in awayName:
                    rank2 = LgtableList.index(a) + 1
                else:
                    pass
            tableRkList = [rank, rank2, len(LgtableList)]


            #goals
            goal_percent = int(((home_GF + away_GA + home_GA + away_GF + h2hs[2])/16)*100)
            goal_percent2 = int(((home_GF5 + away_GA5 + home_GA5 + away_GF5 + h2hs[3])/16)*100)
            #home win
            homeWin_percent = int((((((2 * home_form_value) + (2 * away_form_value) + home_Gen_value)/25) + (0.125 * h2hs[1]))/1.25) * 100)

            #table ranking
            TableDiff = int(((tableRkList[1] - tableRkList[0])/(tableRkList[2] - 1))*100)
            space = ' '

            Values = [country, homeName, awayName, compname, tableRkList[0], tableRkList[1], tableRkList[2], home_form_value, away_form_value, home_Gen_value, h2hs[1], homeWin_percent, TableDiff, space, home_GF, away_GA, home_GA, away_GF, h2hs[2], goal_percent, home_GF5, away_GA5, home_GA5, away_GF5, h2hs[3], goal_percent2 ]
            with open('Match Analysis {x}.csv'.format(x=matchD), 'a', encoding = 'utf-8-sig') as file:
                    writer = csv.writer(file)
                    writer.writerow(Values)
            print(f'Fixture {num077} extracted')

        num077+=1
else:    
    num077 = int(input('Input fixture to start from : '))
    num07 = num077 - 1
    
    with open(f'fixtures {matchD}.txt','r') as fixtures:
        fixturelinks = fixtures.readlines()
       
    fixturelinks = [l.replace('\n','') for l in fixturelinks]
    fixturelinks1 = fixturelinks[num07:]
    print('TOTAL MATCHES FOUND = '+str(len(fixturelinks)))
    print('UNEXTRACTED MATCHES = '+str(len(fixturelinks1)))
        
  
    for match3 in fixturelinks[num07:]:
        #driver.get(match3)

        LgTablePg = requests.get(match3, headers=headers)

        country = match3.split('/')[8].upper() #country
        compname = match3.split('/')[9].replace('-',' ').upper() #league

        
        try:
            Lgtable = pd.read_html(LgTablePg.text, match = 'MP')[0]
            #find elements with class name = team-title
            #teams = driver.find_elements(By.CLASS_NAME, "team-title")
            soup = bs.BeautifulSoup(LgTablePg.content, 'html.parser')
            teAms = soup.find_all("a", {"class":"team-title"})
            
            #get html attribute of found element for home team
            #homeLink = teams[0].get_attribute('outerHTML').split('"')[3]
            homeLink = teAms[0].get('href')
            
            #get html attribute of found element for away team
            #awayLink = teams[1].get_attribute('outerHTML').split('"')[3]
            awayLink = teAms[1].get('href')
            
            #convert to matches pagelink of home and away teams
            homeLink = 'https://ng.soccerway.com'+homeLink+'/matches/'
            awayLink = 'https://ng.soccerway.com'+awayLink+'/matches/'

            #get html of home matches page
            homeLinkH = requests.get(homeLink, headers=headers)
            time.sleep(1)

            #read the first table of the html
            homeDF = pd.read_html(homeLinkH.text)[0]

            #rename columns
            homeDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #rewrite matchDay from yyyy/mm/dd to dd/mm/yy
            dayMatch = matchDay[8:]+matchDay[4:8]+matchDay[2:4]

            #create homeDF1 which is all rows bearing dayMatch as its Date data
            homeDF1 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH
            date_indexH = int(list(homeDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName = homeDF1.loc[date_indexH,'Competition']

            #home & away team names
            homeName = homeDF1.loc[date_indexH,'Home']
            awayName = homeDF1.loc[date_indexH,'Away']

            #cleaning homeDF

            #drop all rows except one with Competition column data = compName 
            homeDF = homeDF[homeDF['Competition'] == compName]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #create homeDF2 which is all rows bearing dayMatch as its Date data
            homeDF2 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH2
            date_indexH2 = int(list(homeDF2['Date'].index)[0])

            #drop all rows including and after matchDay
            homeDF = homeDF.iloc[:date_indexH2]

            #drop all columns except specificied
            homeDF = homeDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_list = homeDF['Score'].values.tolist()
            score_list1 = []
            for a in score_list:
                try:
                    eval(a)
                except:
                    score_list1.append(None)
                else:
                    score_list1.append(eval(a))
            homeDF['Points'] = score_list1

            #reverse rows in homeDF
            homeDF = homeDF.iloc[::-1]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #DF for last 5 matches where home team played at home
            homeDF_home = homeDF[homeDF['Home']==homeName].head(5)

            #DF for last 5 macthes for home
            homeDF_Gen = homeDF.head(5)

            #DF for last 5 matches where home team played at away
            homeDF_away = homeDF[homeDF['Away']==homeName].head(5)


            #homeName = name of home team
            #awayName = name of away team
            #homeDF = DF for all matches for home team
            #compName = Competiton name home POV
            #homeDF_home = home's home analysis
            #homeDF_away = home's away analysis

            #get html of away matches page
            awayLinkH = requests.get(awayLink, headers=headers)

            #read the first table of the html
            awayDF = pd.read_html(awayLinkH.text)[0]

            #rename columns
            awayDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #create awayDF1 which is all rows bearing dayMatch as its Date data
            awayDF1 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA = int(list(awayDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName2 = awayDF1.loc[date_indexA,'Competition']

            #drop all rows except one with Competition column data = compName 
            awayDF = awayDF[awayDF['Competition'] == compName2]

            #reset index
            awayDF = awayDF.reset_index(drop=True)

            #create awayDF2 which is all rows bearing dayMatch as its Date data
            awayDF2 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA2 = int(list(awayDF2['Date'].index)[0])

            #drop all rows including and after macthDay
            awayDF = awayDF.iloc[:date_indexA2]  

            #drop all columns except specificied
            awayDF = awayDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_listA = awayDF['Score'].values.tolist()
            score_listA1 = []
            for a in score_listA:
                try:
                    eval(a)
                except:
                    score_listA1.append(None)
                else:
                    score_listA1.append(eval(a))
            awayDF['Points'] = score_listA1

            #reverse rows in awayDF
            awayDF = awayDF.iloc[::-1]

            #DF for last 5 matches where away team played at home
            awayDF_home = awayDF[awayDF['Home']==awayName].head(5)

            #DF for last 5 matches where away team played at away
            awayDF_away = awayDF[awayDF['Away']==awayName].head(5)


            #awayName = name of away team
            #compName2 = competition name away POV
            #awayDF = DF for matches played by away team
            #awayDF_home = DF for away's home analysis
            #awayDF_away = DF for away's away analysis



            #evaluate home team's home form 

            #convert 'Points' homeDF_home columns to a list
            HPointsList = homeDF_home['Points'].values.tolist()

            #counter
            num001 = 0
            #iterate across list, and change point data into correct point value
            for point in HPointsList:
                if point > 0:
                    HPointsList[num001] = 1
                elif point < 0:
                    HPointsList[num001] = 0
                elif point == 0:
                    HPointsList[num001] = 0.5
                else:
                    HPointsList[num001] = 0
                num001+=1
            #calculate homeform value
            home_form_value = sum(HPointsList)

            #evaluate away team's away form

            #convert 'Points' awayDF_away columns to a list
            APointsList = awayDF_away['Points'].values.tolist()

            #counter
            num003 = 0
            #iterate across list, and change point data into correct point value
            for point in APointsList:
                if point > 0:
                    APointsList[num003] = 1
                elif point < 0:
                    APointsList[num003] = 0
                elif point == 0:
                    APointsList[num003] = 0.5
                else:
                    APointsList[num003] = 0
                num003+=1
            #calculate awayform value
            away_form_value = sum(APointsList)

            #evaluate home team's general form

            #convert 'Points' homeDF_Gen columns to a list
            HGPointsList = homeDF_Gen['Points'].values.tolist()

            #convert 'Home' homeDF_Gen columns to a list
            homeTeamList = homeDF_Gen['Home'].values.tolist()

            #counter
            num002 = 0
            #iterate across list, and change point data into correct point value
            for point in HGPointsList:
                if homeTeamList[num002] == homeName:
                    if point > 0:
                        HGPointsList[num002] = 1
                    elif point < 0:
                        HGPointsList[num002] = 0
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                else:
                    if point > 0:
                        HGPointsList[num002] = 0
                    elif point < 0:
                        HGPointsList[num002] = 1
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                num002+=1
            #calculate homeGen value
            home_Gen_value = sum(HGPointsList)


            #HOME TEAM GOALS

            #convert homeDF_home 'Score' column into list
            homeGoalList = homeDF_home['Score'].values.tolist()

            #check score lines in above homeGoallist and append 1 point (homeGoalF) to scorelines where home scored
            homeGoalF = []
            homeGoalF5 = []
            for goal in homeGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        homeGoalF.append(1)
                    else:
                        homeGoalF.append(0)
                    if int(goal[0]) > 1:
                        homeGoalF5.append(1)
                    else:
                        homeGoalF5.append(0)
            #sum the appended points to get home_GF
            home_GF = sum(homeGoalF)
            home_GF5 = sum(homeGoalF5)
            #check score lines in above homeGoalList and 
            #append 1 point (homeGoalA) to scorelines where home conceded
            homeGoalA = []
            homeGoalA5 = []
            for goal in homeGoalList[:2]:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        homeGoalA.append(1)
                    else:
                        homeGoalA.append(0)
                    if int(goal[4]) > 1:
                        homeGoalA5.append(1)
                    else:
                        homeGoalA5.append(0)
            #sum the appended points to get home_GA
            home_GA = sum(homeGoalA)
            home_GA5 = sum(homeGoalA5)

            #AWAY TEAM GOALS

            #convert awayDF_away 'Score' column into list
            awayGoalList = awayDF_away['Score'].values.tolist()

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalA) to scorelines where away conceded
            awayGoalA = []
            awayGoalA5 = []
            for goal in awayGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        awayGoalA.append(1)
                    else:
                        awayGoalA.append(0)
                    if int(goal[0]) > 1:
                        awayGoalA5.append(1)
                    else:
                        awayGoalA5.append(0)

            #sum the appended points to get away_GA
            away_GA = sum(awayGoalA)
            away_GA5 = sum(awayGoalA5)           

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalF) to scorelines where away scored
            awayGoalF = []
            awayGoalF5 = []
            for goal in awayGoalList[:2]:
                try: 
                    int(goal)
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        awayGoalF.append(1)
                    else:
                        awayGoalF.append(0)
                    if int(goal[4]) > 0:
                        awayGoalF5.append(1)
                    else:
                        awayGoalF5.append(0)
            #sum the appended points to get away_GF
            away_GF = sum(awayGoalF)
            away_GF5 = sum(awayGoalF5)

            # h2h

            #link to h2h
            dink = match3+'head2head'
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                       'AppleWebKit/537.36 (KHTML, like Gecko) '\
                       'Chrome/75.0.3770.80 Safari/537.36'}

            #request h2h page
            pgaa = requests.get(dink, headers=headers)



            ##the table is arranged in anti-chronology, the recent data is on top
            ##and the  match to be played is the first row
            ##which want two most recent h2h data


            #read html page using pandas call it jon
            jon = pd.read_html(pgaa.text)[0]
            #jon

            #reverse rows in jon
            jon = jon.iloc[::-1]
            #jon

            #reset index
            jon = jon.reset_index(drop=True)
            #jon

            #create jon1 which is all rows bearing dayMatch as its Date data
            jon1 = jon[jon['Date'] == dayMatch]

            #get row number of matchDay
            date_indexJ = int(list(jon1['Date'].index)[0])

            #drop all rows including and after the specified index row
            jon = jon.iloc[:date_indexJ]

            #drop all columns except specificied
            jon = jon[['Home team', 'Score/Time', 'Away team']]

            #rename columns
            jon.rename(columns={'Home team':'Home', 'Score/Time': 'Score','Away team': 'Away'}, inplace=True)

            #drop all rows except last 2 rows
            #jon = jon.tail(2)

            jon = jon.tail(2)
            jon = jon.reset_index(drop=True)
            #jon

            try:
                jon.loc[0, 'Home']
            except:
                h2hs = ['Home', 0, 0]
            else:            
                if jon.loc[0,'Home'] == jon.loc[jon.shape[0] - 1,'Home']:
                    result=[]
                    num002 = 0
                    for a in jon['Score']:
                        try:
                            eval(a)
                        except:
                            result.append(-1)
                        else:                
                            gd = int(eval(jon.loc[num002,'Score']))
                            result.append(gd)               
                        num002+=1
                    num003 = 0
                    for b in result:
                        if b > 0:
                            result[num003] = 1
                        elif b < 0:
                            result[num003] = 0
                        else:
                            result[num003] = 0.5
                        num003+=1
                    hn = jon.loc[0, 'Home']
                    result = sum(result)

                else:
                    result=[]
                    try:
                        eval(jon.loc[0,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[0, 'Score']))
                        result.append(gd)
                    try:
                        eval(jon.loc[1,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[1, 'Score'][::-1]))
                        result.append(gd)
                    num004 = 0           
                    for c in result:
                        if c > 0:
                            result[num004] = 1
                        elif c < 0:
                            result[num004] = 0
                        else:
                            result[num004] = 0.5
                        num004+=1
                    result = sum(result)
                    hn = jon.loc[0, 'Home']

                if hn in homeName:
                    pass
                else:
                    result = 2 - result

                h2hgoalist = []
                h2hgoalist5 = []
                for a in jon['Score']:
                    x = a.replace('-','+')
                    try:
                        eval(x)
                    except:
                        h2hgoalist.append(0)
                    else:
                        if int(eval(x)) >= 2:
                            h2hgoalist.append(1)
                        else:
                            h2hgoalist.append(0)
                        if int(eval(x)) > 2:
                            h2hgoalist5.append(1)
                        else:
                            h2hgoalist5.append(0)
                    
                h2h_goalist = sum(h2hgoalist)  
                h2h_goalist5 = sum(h2hgoalist5) 
                h2hs = [hn, result, h2h_goalist, h2h_goalist5]



            LgtableList = Lgtable['Team'].values.tolist()
            rank = 0
            rank2 = 0
            for a in LgtableList:
                if a in homeName:
                    rank = LgtableList.index(a) + 1
                elif a in awayName:
                    rank2 = LgtableList.index(a) + 1
                else:
                    pass
            tableRkList = [rank, rank2, len(LgtableList)]


            #goals
            goal_percent = int(((home_GF + away_GA + home_GA + away_GF + h2hs[2])/16)*100)
            goal_percent2 = int(((home_GF5 + away_GA5 + home_GA5 + away_GF5 + h2hs[3])/16)*100)
            #home win
            homeWin_percent = int((((((2 * home_form_value) + (2 * away_form_value) + home_Gen_value)/25) + (0.125 * h2hs[1]))/1.25) * 100)

            #table ranking
            TableDiff = int(((tableRkList[1] - tableRkList[0])/(tableRkList[2] - 1))*100)
            space = ' '

        except:
            print(f'Fixture {num077} skipped')
            num077+=1
            continue
        else:
            Lgtable = pd.read_html(LgTablePg.text, match = 'MP')[0]
            #find elements with class name = team-title
            #teams = driver.find_elements(By.CLASS_NAME, "team-title")
            soup = bs.BeautifulSoup(LgTablePg.content, 'html.parser')
            teAms = soup.find_all("a", {"class":"team-title"})
            
            #get html attribute of found element for home team
            #homeLink = teams[0].get_attribute('outerHTML').split('"')[3]
            homeLink = teAms[0].get('href')
            
            #get html attribute of found element for away team
            #awayLink = teams[1].get_attribute('outerHTML').split('"')[3]
            awayLink = teAms[1].get('href')
            
            #convert to matches pagelink of home and away teams
            homeLink = 'https://ng.soccerway.com'+homeLink+'/matches/'
            awayLink = 'https://ng.soccerway.com'+awayLink+'/matches/'

            #get html of home matches page
            homeLinkH = requests.get(homeLink, headers=headers)
            time.sleep(1)

            #read the first table of the html
            homeDF = pd.read_html(homeLinkH.text)[0]

            #rename columns
            homeDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #rewrite matchDay from yyyy/mm/dd to dd/mm/yy
            dayMatch = matchDay[8:]+matchDay[4:8]+matchDay[2:4]

            #create homeDF1 which is all rows bearing dayMatch as its Date data
            homeDF1 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH
            date_indexH = int(list(homeDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName = homeDF1.loc[date_indexH,'Competition']

            #home & away team names
            homeName = homeDF1.loc[date_indexH,'Home']
            awayName = homeDF1.loc[date_indexH,'Away']

            #cleaning homeDF

            #drop all rows except one with Competition column data = compName 
            homeDF = homeDF[homeDF['Competition'] == compName]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #create homeDF2 which is all rows bearing dayMatch as its Date data
            homeDF2 = homeDF[homeDF['Date'] == dayMatch]

            #rertrieve row index for matchDay and assign to variable date_indexH2
            date_indexH2 = int(list(homeDF2['Date'].index)[0])

            #drop all rows including and after matchDay
            homeDF = homeDF.iloc[:date_indexH2]

            #drop all columns except specificied
            homeDF = homeDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_list = homeDF['Score'].values.tolist()
            score_list1 = []
            for a in score_list:
                try:
                    eval(a)
                except:
                    score_list1.append(None)
                else:
                    score_list1.append(eval(a))
            homeDF['Points'] = score_list1

            #reverse rows in homeDF
            homeDF = homeDF.iloc[::-1]

            #reset index location
            homeDF = homeDF.reset_index(drop=True)

            #DF for last 5 matches where home team played at home
            homeDF_home = homeDF[homeDF['Home']==homeName].head(5)

            #DF for last 5 macthes for home
            homeDF_Gen = homeDF.head(5)

            #DF for last 5 matches where home team played at away
            homeDF_away = homeDF[homeDF['Away']==homeName].head(5)


            #homeName = name of home team
            #awayName = name of away team
            #homeDF = DF for all matches for home team
            #compName = Competiton name home POV
            #homeDF_home = home's home analysis
            #homeDF_away = home's away analysis

            #get html of away matches page
            awayLinkH = requests.get(awayLink, headers=headers)

            #read the first table of the html
            awayDF = pd.read_html(awayLinkH.text)[0]

            #rename columns
            awayDF.rename(columns={'Home team':'Score', 'Score/Time': 'Away','Outcome': 'Home'}, inplace=True)

            #create awayDF1 which is all rows bearing dayMatch as its Date data
            awayDF1 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA = int(list(awayDF1['Date'].index)[0])

            #get the competiton data of the first row of homeDF1 assign to variable called compName
            compName2 = awayDF1.loc[date_indexA,'Competition']

            #drop all rows except one with Competition column data = compName 
            awayDF = awayDF[awayDF['Competition'] == compName2]

            #reset index
            awayDF = awayDF.reset_index(drop=True)

            #create awayDF2 which is all rows bearing dayMatch as its Date data
            awayDF2 = awayDF[awayDF['Date'] == dayMatch]

            #get row number of matchDay
            date_indexA2 = int(list(awayDF2['Date'].index)[0])

            #drop all rows including and after macthDay
            awayDF = awayDF.iloc[:date_indexA2]  

            #drop all columns except specificied
            awayDF = awayDF[['Home', 'Score', 'Away']]

            #calculate points and append as column
            score_listA = awayDF['Score'].values.tolist()
            score_listA1 = []
            for a in score_listA:
                try:
                    eval(a)
                except:
                    score_listA1.append(None)
                else:
                    score_listA1.append(eval(a))
            awayDF['Points'] = score_listA1

            #reverse rows in awayDF
            awayDF = awayDF.iloc[::-1]

            #DF for last 5 matches where away team played at home
            awayDF_home = awayDF[awayDF['Home']==awayName].head(5)

            #DF for last 5 matches where away team played at away
            awayDF_away = awayDF[awayDF['Away']==awayName].head(5)


            #awayName = name of away team
            #compName2 = competition name away POV
            #awayDF = DF for matches played by away team
            #awayDF_home = DF for away's home analysis
            #awayDF_away = DF for away's away analysis



            #evaluate home team's home form 

            #convert 'Points' homeDF_home columns to a list
            HPointsList = homeDF_home['Points'].values.tolist()

            #counter
            num001 = 0
            #iterate across list, and change point data into correct point value
            for point in HPointsList:
                if point > 0:
                    HPointsList[num001] = 1
                elif point < 0:
                    HPointsList[num001] = 0
                elif point == 0:
                    HPointsList[num001] = 0.5
                else:
                    HPointsList[num001] = 0
                num001+=1
            #calculate homeform value
            home_form_value = sum(HPointsList)

            #evaluate away team's away form

            #convert 'Points' awayDF_away columns to a list
            APointsList = awayDF_away['Points'].values.tolist()

            #counter
            num003 = 0
            #iterate across list, and change point data into correct point value
            for point in APointsList:
                if point > 0:
                    APointsList[num003] = 1
                elif point < 0:
                    APointsList[num003] = 0
                elif point == 0:
                    APointsList[num003] = 0.5
                else:
                    APointsList[num003] = 0
                num003+=1
            #calculate awayform value
            away_form_value = sum(APointsList)

            #evaluate home team's general form

            #convert 'Points' homeDF_Gen columns to a list
            HGPointsList = homeDF_Gen['Points'].values.tolist()

            #convert 'Home' homeDF_Gen columns to a list
            homeTeamList = homeDF_Gen['Home'].values.tolist()

            #counter
            num002 = 0
            #iterate across list, and change point data into correct point value
            for point in HGPointsList:
                if homeTeamList[num002] == homeName:
                    if point > 0:
                        HGPointsList[num002] = 1
                    elif point < 0:
                        HGPointsList[num002] = 0
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                else:
                    if point > 0:
                        HGPointsList[num002] = 0
                    elif point < 0:
                        HGPointsList[num002] = 1
                    elif point == 0:
                        HGPointsList[num002] = 0.5
                    else:
                        HGPointsList[num002] = 0
                num002+=1
            #calculate homeGen value
            home_Gen_value = sum(HGPointsList)


            #HOME TEAM GOALS

            #convert homeDF_home 'Score' column into list
            homeGoalList = homeDF_home['Score'].values.tolist()

            #check score lines in above homeGoallist and append 1 point (homeGoalF) to scorelines where home scored
            homeGoalF = []
            homeGoalF5 = []
            for goal in homeGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        homeGoalF.append(1)
                    else:
                        homeGoalF.append(0)
                    if int(goal[0]) > 1:
                        homeGoalF5.append(1)
                    else:
                        homeGoalF5.append(0)
            #sum the appended points to get home_GF
            home_GF = sum(homeGoalF)
            home_GF5 = sum(homeGoalF5)
            #check score lines in above homeGoalList and 
            #append 1 point (homeGoalA) to scorelines where home conceded
            homeGoalA = []
            homeGoalA5 = []
            for goal in homeGoalList[:2]:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        homeGoalA.append(1)
                    else:
                        homeGoalA.append(0)
                    if int(goal[4]) > 1:
                        homeGoalA5.append(1)
                    else:
                        homeGoalA5.append(0)
            #sum the appended points to get home_GA
            home_GA = sum(homeGoalA)
            home_GA5 = sum(homeGoalA5)

            #AWAY TEAM GOALS

            #convert awayDF_away 'Score' column into list
            awayGoalList = awayDF_away['Score'].values.tolist()

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalA) to scorelines where away conceded
            awayGoalA = []
            awayGoalA5 = []
            for goal in awayGoalList:
                try:
                    int(goal[0])
                except:
                    continue
                else:
                    if int(goal[0]) > 0:
                        awayGoalA.append(1)
                    else:
                        awayGoalA.append(0)
                    if int(goal[0]) > 1:
                        awayGoalA5.append(1)
                    else:
                        awayGoalA5.append(0)

            #sum the appended points to get away_GA
            away_GA = sum(awayGoalA)
            away_GA5 = sum(awayGoalA5)           

            #check score lines in above awayGoallist and 
            #append 1 point (awayGoalF) to scorelines where away scored
            awayGoalF = []
            awayGoalF5 = []
            for goal in awayGoalList[:2]:
                try: 
                    int(goal)
                except:
                    continue
                else:
                    if int(goal[4]) > 0:
                        awayGoalF.append(1)
                    else:
                        awayGoalF.append(0)
                    if int(goal[4]) > 0:
                        awayGoalF5.append(1)
                    else:
                        awayGoalF5.append(0)
            #sum the appended points to get away_GF
            away_GF = sum(awayGoalF)
            away_GF5 = sum(awayGoalF5)

            # h2h

            #link to h2h
            dink = match3+'head2head'
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                       'AppleWebKit/537.36 (KHTML, like Gecko) '\
                       'Chrome/75.0.3770.80 Safari/537.36'}

            #request h2h page
            pgaa = requests.get(dink, headers=headers)



            ##the table is arranged in anti-chronology, the recent data is on top
            ##and the  match to be played is the first row
            ##which want two most recent h2h data


            #read html page using pandas call it jon
            jon = pd.read_html(pgaa.text)[0]
            #jon

            #reverse rows in jon
            jon = jon.iloc[::-1]
            #jon

            #reset index
            jon = jon.reset_index(drop=True)
            #jon

            #create jon1 which is all rows bearing dayMatch as its Date data
            jon1 = jon[jon['Date'] == dayMatch]

            #get row number of matchDay
            date_indexJ = int(list(jon1['Date'].index)[0])

            #drop all rows including and after the specified index row
            jon = jon.iloc[:date_indexJ]

            #drop all columns except specificied
            jon = jon[['Home team', 'Score/Time', 'Away team']]

            #rename columns
            jon.rename(columns={'Home team':'Home', 'Score/Time': 'Score','Away team': 'Away'}, inplace=True)

            #drop all rows except last 2 rows
            #jon = jon.tail(2)

            jon = jon.tail(2)
            jon = jon.reset_index(drop=True)
            #jon

            try:
                jon.loc[0, 'Home']
            except:
                h2hs = ['Home', 0, 0]
            else:            
                if jon.loc[0,'Home'] == jon.loc[jon.shape[0] - 1,'Home']:
                    result=[]
                    num002 = 0
                    for a in jon['Score']:
                        try:
                            eval(a)
                        except:
                            result.append(-1)
                        else:                
                            gd = int(eval(jon.loc[num002,'Score']))
                            result.append(gd)               
                        num002+=1
                    num003 = 0
                    for b in result:
                        if b > 0:
                            result[num003] = 1
                        elif b < 0:
                            result[num003] = 0
                        else:
                            result[num003] = 0.5
                        num003+=1
                    hn = jon.loc[0, 'Home']
                    result = sum(result)

                else:
                    result=[]
                    try:
                        eval(jon.loc[0,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[0, 'Score']))
                        result.append(gd)
                    try:
                        eval(jon.loc[1,'Score'])
                    except:
                        result.append(-1)
                    else:
                        gd = int(eval(jon.loc[1, 'Score'][::-1]))
                        result.append(gd)
                    num004 = 0           
                    for c in result:
                        if c > 0:
                            result[num004] = 1
                        elif c < 0:
                            result[num004] = 0
                        else:
                            result[num004] = 0.5
                        num004+=1
                    result = sum(result)
                    hn = jon.loc[0, 'Home']

                if hn in homeName:
                    pass
                else:
                    result = 2 - result

                h2hgoalist = []
                h2hgoalist5 = []
                for a in jon['Score']:
                    x = a.replace('-','+')
                    try:
                        eval(x)
                    except:
                        h2hgoalist.append(0)
                    else:
                        if int(eval(x)) >= 2:
                            h2hgoalist.append(1)
                        else:
                            h2hgoalist.append(0)
                        if int(eval(x)) > 2:
                            h2hgoalist5.append(1)
                        else:
                            h2hgoalist5.append(0)
                    
                h2h_goalist = sum(h2hgoalist)  
                h2h_goalist5 = sum(h2hgoalist5)  
                h2hs = [hn, result, h2h_goalist, h2h_goalist5]

            

            LgtableList = Lgtable['Team'].values.tolist()
            rank = 0
            rank2 = 0
            for a in LgtableList:
                if a in homeName:
                    rank = LgtableList.index(a) + 1
                elif a in awayName:
                    rank2 = LgtableList.index(a) + 1
                else:
                    pass
            tableRkList = [rank, rank2, len(LgtableList)]


            #goals
            goal_percent = int(((home_GF + away_GA + home_GA + away_GF + h2hs[2])/16)*100)
            goal_percent2 = int(((home_GF5 + away_GA5 + home_GA5 + away_GF5 + h2hs[3])/16)*100)
            #home win
            homeWin_percent = int((((((2 * home_form_value) + (2 * away_form_value) + home_Gen_value)/25) + (0.125 * h2hs[1]))/1.25) * 100)

            #table ranking
            TableDiff = int(((tableRkList[1] - tableRkList[0])/(tableRkList[2] - 1))*100)
            space = ' '

            Values = [country, homeName, awayName, compname, tableRkList[0], tableRkList[1], tableRkList[2], home_form_value, away_form_value, home_Gen_value, h2hs[1], homeWin_percent, TableDiff, space, home_GF, away_GA, home_GA, away_GF, h2hs[2], goal_percent, home_GF5, away_GA5, home_GA5, away_GF5, h2hs[3],goal_percent2 ]
            with open('Match Analysis {x}.csv'.format(x=matchD), 'a', encoding = 'utf-8-sig') as file:
                writer = csv.writer(file)
                writer.writerow(Values)
            print(f'Fixture {num077} extracted')
        
        num077+=1
print('ALL FIXTURES EXTRACTED')

Input Match Date (YYYY/MM/DD):2023/04/21
Input 0 to start from beginning : 0
TOTAL MATCHES FOUND = 540
Fixture 1 extracted
Fixture 2 extracted
Fixture 3 extracted
Fixture 4 extracted
Fixture 5 extracted
Fixture 6 extracted
Fixture 7 extracted
Fixture 8 extracted
Fixture 9 skipped
Fixture 10 skipped
Fixture 11 skipped
Fixture 12 extracted
Fixture 13 extracted
Fixture 14 skipped
Fixture 15 extracted
Fixture 16 extracted
Fixture 17 skipped
Fixture 18 extracted
Fixture 19 extracted
Fixture 20 extracted
Fixture 21 extracted
Fixture 22 extracted
Fixture 23 extracted
Fixture 24 extracted
Fixture 25 skipped
Fixture 26 extracted
Fixture 27 extracted
Fixture 28 extracted
Fixture 29 extracted
Fixture 30 extracted
Fixture 31 extracted
Fixture 32 extracted
Fixture 33 extracted
Fixture 34 extracted
Fixture 35 extracted
Fixture 36 extracted
Fixture 37 extracted
Fixture 38 extracted
Fixture 39 extracted
Fixture 40 extracted
Fixture 41 skipped
Fixture 42 extracted
Fixture 43 extracted
Fixture 44 extrac

Fixture 378 skipped
Fixture 379 skipped
Fixture 380 skipped
Fixture 381 skipped
Fixture 382 skipped
Fixture 383 skipped
Fixture 384 skipped
Fixture 385 extracted
Fixture 386 extracted
Fixture 387 extracted
Fixture 388 extracted
Fixture 389 extracted
Fixture 390 extracted
Fixture 391 extracted
Fixture 392 extracted
Fixture 393 extracted
Fixture 394 extracted
Fixture 395 extracted
Fixture 396 extracted
Fixture 397 extracted
Fixture 398 extracted
Fixture 399 extracted
Fixture 400 extracted
Fixture 401 extracted
Fixture 402 extracted
Fixture 403 skipped
Fixture 404 skipped
Fixture 405 skipped
Fixture 406 skipped
Fixture 407 extracted
Fixture 408 extracted
Fixture 409 extracted
Fixture 410 skipped
Fixture 411 extracted
Fixture 412 extracted
Fixture 413 extracted
Fixture 414 extracted
Fixture 415 extracted
Fixture 416 extracted
Fixture 417 extracted
Fixture 418 extracted
Fixture 419 extracted
Fixture 420 extracted
Fixture 421 extracted
Fixture 422 extracted
Fixture 423 extracted
Fixture 424 

###### num077

In [ ]:
'716matches = 1207mb'
'1021matches = 1113mb'

In [ ]:
'2023/04/16'